# Building A Serverless Multimodal ChatBot: Part 1
--------------------------------------------------

## Introduction
In this blog post I will go over how to create a create multimodal chatbot using [Large Language Models (LLM)](https://en.wikipedia.org/wiki/Large_language_model). Specifically, we'll build an app that you can submit a prompt using speech and get the bot's reply back as speech. The conversation will be transcribed to text we can read it, but also so that we can interact with the LLM. I will go over how to do this all in a serverless framework and APIs so that (baring the app getting really popular) the costs will be next to nothing! We'll do this by using [LangChain](https://www.langchain.com/) & [Groq API](https://groq.com/) to interact with the [Llama 3](https://ai.meta.com/blog/meta-llama-3/) Open Source LLM. Then We'll use the Google Cloud API for [Text-To-Speech](https://cloud.google.com/text-to-speech?hl=en) and [Speech-To-Text](https://cloud.google.com/speech-to-text/?hl=en). For production and deployment we'll use [Streamlit](https://streamlit.io/), [Docker](https://www.docker.com/) and [Google Cloud Run](https://cloud.google.com/run).

Lastly, I wanted to make this app so that my wife could practice Hebrew with and my mom could practice French with, so I made the app be able to be multilingual. In this post I'll focus on building the app and running it locally, while in a follow up one I will make a post on how to deploy the app.

Now let's go over how to use LLMs!

### Chatting With Llama 3 Using LangChain & Groq

There are many different [Large Language Models (LLM)](https://en.wikipedia.org/wiki/Large_language_model) that we can use for this app, but I chose [Llama 3](https://ai.meta.com/blog/meta-llama-3/) since its Open Source (free), specifically, I used the [Llama 3.3 70 Billion parameter model](https://groq.com/a-new-scaling-paradigm-metas-llama-3-3-70b-challenges-death-of-scaling-law/).

For serving the model I used the [Groq API](https://groq.com/) since its free (up until a point). There are quite a few methods to interact with Groq and I chose to use [LangChain](https://www.langchain.com/). At first I thought LangChaing was a little over engineered (why do you need class for templated prompts? Isnt it just an f-string?), but now I am on-board! The API is super-powerful! It allows for a consistent API across most models and abstracts away a lot painpoints. The prompt templates do make sense now, and my only complaint is I cant tell what library something should come from (langchain, langchain_core, langchain_community?), but given ho much the API has changed around, it seems neither does the community. :-)

The first thing I'll do is import [ChatGroq](https://python.langchain.com/docs/integrations/chat/groq/) class and use [pydot-env](https://pypi.org/project/python-dotenv/) to help with environment variables that hold my API keys.



In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

Instantiating the ChatGroq chat object gives me a model I can query using the `invoke` method:

In [3]:
llm = ChatGroq(
        model="llama-3.3-70b-versatile",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2)


In [5]:
result = llm.invoke("What is the square root of 9?")

The returned object is of type [AIMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html) and the message can be obtained with the `.content` attribute:

In [6]:
print(result.content)

The square root of 9 is 3.


Simple enough! Now lets go over [PrompteTemplate](https://python.langchain.com/docs/concepts/prompt_templates/) in LangChain. PromptTemplates allow us to create prompts (question/queries) that can have variables in them (like [f-strings](https://realpython.com/python-f-strings/)). This allows us to chain together our prompt with the LLM into pipelines called "Chains" so that all we have to do is invoke the chain with a dictionary with the variable values and we will get back out answer for that invocation's prompt values!

Let's show how TemplatePrompts work and how to use them with LLMs as a chain. First we import the PromptTemplate class

In [7]:
from langchain_core.prompts import PromptTemplate

Next we create a string that looks like an `f-string`

In [8]:
template = "What is the square root of {n}?"

Now we use the [from_template](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/prompts/prompt.py#L249) class method (pretty cool to see a class method, I have really not seen it used that often!) to make a templated prompt:

In [12]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={}, template='What is the square root of {n}?')

Now we can create our prompt by filling in the variable `n` using a dictionary and the `invoke` method on the the prompt

In [15]:
prompt.invoke({"n": 9})

StringPromptValue(text='What is the square root of 9?')

Now the really cool thing is when we chain the PromptTemplate and the LLM together into a "Chain" using the `|` to represent seperate components of the chain:

In [17]:
chain = prompt | llm

We can go from value of n=16 to the answer now with just the invoke command using a dictionary as input:

In [18]:
result = chain.invoke({"n": 16})
print(result.content)

The square root of 16 is 4.


Great!

Now we can put it all together to create a function that takes a message in one language and converts it into another.

In [19]:
def translate_text(language: str, text: str) -> str:
        if language not in ("English", "French", "Hebrew"):
                raise ValueError(f"Not valid language choice: {language}")
        
        template = "Translate the following into {language} and only return the translated text: {text}"

        prompt = PromptTemplate.from_template(template)

        llm = ChatGroq(
                model="llama-3.3-70b-versatile",
                temperature=0,
                max_tokens=None,
                timeout=None,
                max_retries=2)

        translation_chain = prompt | llm 

        result = translation_chain.invoke(
                {
                        "language": language,
                        "text": text,
                }
        )

        return result.content

Now trying it out!

In [20]:
result = translate_text(language="French", text="Hello World!")

In [21]:
print(result)

Bonjour le monde !


Now one thing we have to do is add a bit a memory to our LLM since it wont remember anything we asked previous! See the example below:

In [25]:
print(llm.invoke("Set x = 9").content)
print(llm.invoke("What is x + 3?").content)

x = 9
To determine the value of x + 3, I would need to know the value of x. Could you please provide the value of x?


The LLM has no recollection of anything from prior invocations! At first I thought memory was somthing special, but its really keeping track of the conversation and feeding it into the LLM before asking another question. The chat history will look like list of tuples where the first entry to the tuple signifies whether it is the "ai" system or the "human" and the second entry in the tuple is the message. For examplle the conversation above could be seen as,

    history = [
        ("human", "Set x = 9"),
        ("ai", "9"),
        ("human", "What is x + 3?"),
        ...
    ]

Similar to the [PrompteTemplate](https://python.langchain.com/docs/concepts/prompt_templates/) there is a [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) that can be used to create the history of the chat conversation. This used in conjunction with the [MessagePlaceholder](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html) to unwind the conversation into a prompt with the entire history and the new question at the very end. 

An examle is below:

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("history"),
        ("human", "{question}")
    ]
)

In [31]:
history = [("human", "Set x = 9"), ("ai", "9")]

In [35]:
prompt.invoke(
    {
        "history": history,
        "question": "What is x + 3?"
    }
).messages

[HumanMessage(content='Set x = 9', additional_kwargs={}, response_metadata={}),
 AIMessage(content='9', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is x + 3?', additional_kwargs={}, response_metadata={})]

Now we can form a chain with memory:

In [40]:
history = []
chain = prompt | llm

question = "set x = 9"
answer = chain.invoke({"history": history, "question": question}).content
history.extend([("human", question), ("ai", answer)])

question = "what is x + 3?"
print(chain.invoke({"history": history, "question": question}).content)           

To find x + 3, we need to add 3 to the current value of x, which is 9.

x + 3 = 9 + 3
x + 3 = 12

So, x + 3 is 12.


Now I can put it all together into a function below using the history from above:

In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, System 
from typing import Iterator, List, Tuple

def ask_question(
    history: List[Tuple[str, str]], 
    question: str,
    ai_language: str
) -> str:
    
    llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2)
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", f"""You are a helpful teacher having a conversation with a student in {ai_language}.
             Only reply back in {ai_language} not matter what language the student uses."""),
            MessagesPlaceholder("history"),
            ("human", "{question}")
        ]
    )

    chain = prompt | llm 
    
    response = chain.invoke(
                    {
                        "history": history,
                        "question": question
                    }
    )
    
    answer = response.content

    return answer

In [43]:
print(
    ask_question(
    history=history,
    ai_language="English",
    question="What is x + 3?"
))

To find the value of x + 3, we need to add 3 to the value of x. Since x = 9, we can calculate it as follows:

x + 3 = 9 + 3
= 12

So, x + 3 is equal to 12.


Now the prompt I set in prepending the history allows me to get the answer in any language! For instance,


In [44]:
answer = ask_question(
    history=history,
    ai_language="French",
    question="What is x + 3?")

print(answer)

Pour trouver la valeur de x + 3, il faut ajouter 3 à la valeur de x. Puisque x = 9, on a x + 3 = 9 + 3 = 12. La réponse est donc 12.


Now in Hebrew!

In [45]:
print(translate_text(language="Hebrew", text=answer))

כדי למצוא את הערך של x + 3, יש להוסיף 3 לערך של x. מאחר ש-x = 9, יש לנו x + 3 = 9 + 3 = 12. לכן, התשובה היא 12.


Very cool!! LangChain makes this so easy!

In [3]:
import IPython.display as ipd
ipd.Audio("audio_file.wav")

### Text To Speech & Speech To Text With Google API

### Putting It Together As An App Using Streamlit

### Next Steps

For mapping to a website: https://www.youtube.com/watch?v=lDtvpUYAFzA